## 3.3.1 맥락과 타깃
- 신경망에 '맥락'을 입력했을때 '타깃'이 출현할 확률을 높이는 것

In [1]:
import sys
sys.path.append('..')
from common.util import preprocess

text = 'You say goodbye and I say hello.'
corpus, word_to_id, id_to_word = preprocess(text)

print(corpus)

[0 1 2 3 4 1 5 6]


In [2]:
word_to_id

{'you': 0, 'say': 1, 'goodbye': 2, 'and': 3, 'i': 4, 'hello': 5, '.': 6}

In [9]:
# 단어ID배열인 corpus로부터 맥락과 타깃을 만드는 함수
import sys
sys.path.append('..')
import os
from common.np import *

def create_contexts_target(corpus, window_size=1):
    '''맥락과 타깃 생성
    :param corpus: 말뭉치(단어 ID 목록)
    :param window_size: 윈도우 크기(윈도우 크기가 1이면 타깃 단어 좌우 한 단어씩이 맥락에 포함)
    :return:
    '''
    target = corpus[window_size:-window_size]
    contexts = []

    for idx in range(window_size, len(corpus)-window_size):
        cs = []
        for t in range(-window_size, window_size + 1):
            if t == 0:
                continue
            cs.append(corpus[idx + t])
        contexts.append(cs)

    return np.array(contexts), np.array(target)

In [10]:
corpus

array([0, 1, 2, 3, 4, 1, 5, 6])

In [11]:
corpus[1:-1]   # corpus[window_size:-window_size]

array([1, 2, 3, 4, 1, 5])

In [14]:
# 함수 사용
contexts, target = create_contexts_target(corpus, window_size=1)

print('contexts\n',contexts)
print('target\n',target)

contexts
 [[0 2]
 [1 3]
 [2 4]
 [3 1]
 [4 5]
 [1 6]]
target
 [1 2 3 4 1 5]


- 이 맥락과 타깃의 각 원소는 단어ID이기 때문에, 원핫으로 변환해야함.

## 3.3.2 원핫으로 표현


In [15]:
def convert_one_hot(corpus, vocab_size):
    '''원핫 표현으로 변환
    :param corpus: 단어 ID 목록(1차원 또는 2차원 넘파이 배열)
    :param vocab_size: 어휘 수
    :return: 원핫 표현(2차원 또는 3차원 넘파이 배열)
    '''
    N = corpus.shape[0]

    if corpus.ndim == 1:
        one_hot = np.zeros((N, vocab_size), dtype=np.int32)
        for idx, word_id in enumerate(corpus):
            one_hot[idx, word_id] = 1

    elif corpus.ndim == 2:
        C = corpus.shape[1]
        one_hot = np.zeros((N, C, vocab_size), dtype=np.int32)
        for idx_0, word_ids in enumerate(corpus):
            for idx_1, word_id in enumerate(word_ids):
                one_hot[idx_0, idx_1, word_id] = 1

    return one_hot

In [17]:
corpus

array([0, 1, 2, 3, 4, 1, 5, 6])

In [18]:
corpus.shape

(8,)

In [19]:
corpus.shape[0]

8

In [22]:
corpus.ndim

1

In [23]:
# 데이터 준비과정 한 데 모아 정리

import sys
sys.path.append('..')
from common.util import preprocess, create_contexts_target, convert_one_hot

text = 'You say goodbye and I say hello.'
corpus, word_to_id, id_to_word = preprocess(text)

contexts, target = create_contexts_target(corpus, window_size=1)

vocab_size = len(word_to_id)
target = convert_one_hot(target, vocab_size)
contexts = convert_one_hot(contexts, vocab_size)